In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('cse_courses_1.csv')

In [10]:
df.head(10)

,Year,Term,Course Code,Course Title,Credits,Course Teacher,Sunday,Monday,Tuesday,Wednesday,Thursday
0,1,1,071402CSE1100,Computer Fundamentals Laboratory,1.5,Lecturer Md. Farhan Sadique,2:00 PM - 5:00 PM,NaN,NaN,2:00 PM - 5:00 PM,NaN
1,1,1,071402CSE1101,Structured Programming,3.0,Associate Professor Dr. Manishankar Mondal,NaN,11:00 AM - 12:00 PM,9:00 AM - 10:00 AM,NaN,11:00 AM - 1:00 PM
2,1,1,071402CSE1102,Structured Programming Laboratory,1.5,Associate Professor Dr. Manishankar Mondal,2:00 PM - 5:00 PM,NaN,2:00 PM - 5:00 PM,NaN,NaN
3,1,1,071402CSE1103,Discrete Mathematics,3.0,Professor Dr. Abu Shamim Mohammad Arif,10:00 AM - 11:00 AM,NaN,9:00 AM - 11:00 AM,NaN,9:00 AM - 10:00 PM
4,1,1,054102Math1151,Calculus,3.0,Professor Dr. Sarder Firoz Ahmmed,11:00 AM - 1:00 PM,9:00 AM - 10:00 AM,NaN,NaN,9:00 AM - 10:00 AM
5,1,1,053302Phy1151,Physics,3.0,Associate Professor Md. Shohel Parvez,9:00 AM - 10:00 AM,NaN,12:00 PM - 1:00 PM,9:00 AM - 11:00 AM,NaN
6,1,1,053302Phy1152,Physics Laboratory,1.0,Associate Professor Md. Shohel Parvez,NaN,2:00 PM - 5:00 PM,NaN,NaN,2:00 PM - 5:00 PM
7,1,1,023102Eng1151,English,2.0,Professor Dr. Abdur Rahman Shahin,11:00 AM - 12:00 AM,NaN,NaN,10:00 AM - 12:00 PM,NaN
8,1,1,023102Eng1152,English Skills Laboratory,1.0,Professor Dr. Abdur Rahman Shahin,NaN,2:00 PM - 5:00 PM,NaN,NaN,2:00 PM - 5:00 PM
9,1,2,071402CSE1200,Structured Programming Project,1.5,Associate Professor Dr. Manishankar Mondal,NaN,NaN,2:00 PM - 5:00 PM,2:00 PM - 5:00 PM,NaN


In [11]:
term1_courses = df[df['Term'] == 1]
term2_courses = df[df['Term'] == 2]

In [12]:
import re
import pandas as pd
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

In [13]:
def generate_routine_pdf(df, filename):
    '''
    Generates a PDF routine schedule using provided DataFrame.
    Adopts a session-based scheduling strategy with backtracking,
    ensuring teachers get proper load and labs stay in one day.
    '''
    
    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday"]
    time_slots = ["9–10", "10–11", "11–12", "12–1", "1–2", "2–3", "3–4", "4–5"]
    years = [1, 2, 3, 4]
    
    # Mapping real time strings to slot indices
    time_mapping = {
        "9:00 AM - 10:00 AM": [0],
        "9:00 AM - 11:00 AM": [0, 1],
        "10:00 AM - 11:00 AM": [1],
        "10:00 AM - 12:00 PM": [1, 2],
        "11:00 AM - 12:00 PM": [2],
        "11:00 AM - 1:00 PM": [2, 3],
        "12:00 PM - 1:00 PM": [3],
        "1:00 PM - 2:00 PM": [4],
        "2:00 PM - 5:00 PM": [5, 6, 7],
    }

    def get_teacher_code(full_name):
        clean = re.sub(r'\b(Dr|Professor|Assistant|Associate|Lecturer|Mr|Ms|Mrs)\b\.?', '', full_name, flags=re.I)
        words = clean.strip().split()
        return ''.join([word[0].upper() for word in words if word])

    def get_designation_priority(name):
        if "Professor" in name and "Associate" not in name:
            return 1
        elif "Associate Professor" in name:
            return 2
        elif "Lecturer" in name:
            return 3
        else:
            return 4

    def convert_time_to_slot(time_str):
        return time_mapping.get(time_str, [])

    # Initialize routine
    routine = {day: {y: [""] * len(time_slots) for y in years} for day in weekdays}

    # Track how many credits a teacher is assigned
    teacher_credit_count = {}

    # Create session list
    sessions = []
    for idx, row in df.iterrows():
        for day in weekdays:
            if pd.notna(row[day]):
                teacher = row["Course Teacher"]
                teacher_code = get_teacher_code(teacher)
                credits = row["Credits"]

                # Add session to list
                sessions.append({
                    "day": day,
                    "year": row["Year"],
                    "course_code": row["Course Code"],
                    "course_title": row["Course Title"],
                    "credits": credits,
                    "remaining_credit": credits,  # ← ADD THIS LINE
                    "teacher_name": teacher,
                    "teacher_code": teacher_code,
                    "priority": get_designation_priority(teacher),
                    "time_str": row[day],
                    "slots": convert_time_to_slot(row[day])
                })

    # Sort sessions by teacher priority (lower number = higher priority)
    sessions.sort(key=lambda x: (x["priority"], -x["credits"]))

    # Helper: Check for conflicts
    def has_conflict(day, year, slots, teacher_code):
        for slot in slots:
            if routine[day][year][slot] != "":
                existing_teacher = routine[day][year][slot].split("\n")[1]
                if existing_teacher == teacher_code:
                    return True
        return False

    # Backtracking to assign sessions
    def backtrack(idx):
        if idx == len(sessions):
            return True

        session = sessions[idx]

        # If no credits left, skip
        if session['remaining_credit'] <= 0:
            return backtrack(idx + 1)

        day = session["day"]
        year = session["year"]
        slots = session["slots"]
        teacher_code = session["teacher_code"]
        entry = f"{session['course_code']}\n{teacher_code}"
        is_lab = len(slots) > 2

        # First, try to assign the current teacher's slots without shifting any previous teacher
        if not has_conflict(day, year, slots, teacher_code):
            # Assign the class
            for slot in slots:
                routine[day][year][slot] = entry

            # For lab, deduct all at once and mark it done after day
            if is_lab:
                session['remaining_credit'] = 0
            else:
                session['remaining_credit'] -= len(slots)

            # Recurse to attempt to assign the next session
            if backtrack(idx + 1):
                return True

            # Undo the assignment
            for slot in slots:
                routine[day][year][slot] = ""

            # Undo credit deduction
            if is_lab:
                session['remaining_credit'] = session['credits']
            else:
                session['remaining_credit'] += len(slots)

        # If the current teacher still has remaining credits, only then attempt to shift previous teachers
        if session['remaining_credit'] > 0:
            for prev_teacher_idx in range(idx - 1, -1, -1):  # Check previous teachers
                prev_session = sessions[prev_teacher_idx]

                # Check if previous teacher has remaining credits and can be shifted
                if prev_session['remaining_credit'] > 0:
                    prev_slots = prev_session["slots"]
                    prev_teacher_code = prev_session["teacher_code"]

                    # Check if there's available space for the previous teacher
                    if not has_conflict(day, year, prev_slots, prev_teacher_code):
                        # Move the previous teacher to the current teacher's available slots
                        for prev_slot in prev_slots:
                            routine[day][year][prev_slot] = f"{prev_session['course_code']}\n{prev_teacher_code}"

                        prev_session['remaining_credit'] = 0  # Previous teacher is moved
                        
                        # Recurse to try filling current teacher's remaining slots
                        if backtrack(idx + 1):
                            return True

                        # Undo the move of the previous teacher
                        for prev_slot in prev_slots:
                            routine[day][year][prev_slot] = ""

                        prev_session['remaining_credit'] = prev_session['credits']

        return False

    if not backtrack(0):
        print("Error: Conflict detected or teacher overload.")

    # Building the table
    table_data = [["Day", "Year"] + time_slots]
    for day in weekdays:
        first = True
        for y in years:
            year_str = f"{y}st Year" if y == 1 else f"{y}nd Year" if y == 2 else f"{y}rd Year" if y == 3 else f"{y}th Year"
            row = [day if first else "", year_str]
            row += routine[day][y]
            table_data.append(row)
            first = False

    # Creating PDF
    doc = SimpleDocTemplate(filename, pagesize=(900, 1000), rightMargin=20, leftMargin=20, topMargin=30, bottomMargin=20)
    col_widths = [60, 60] + [80] * len(time_slots)
    table = Table(table_data, colWidths=col_widths, repeatRows=1)

    style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
                        ('FONTSIZE', (0, 0), (-1, -1), 8),
                        ('GRID', (0, 0), (-1, -1), 0.25, colors.black)])

    for i in range(1, len(table_data), 4):
        style.add('SPAN', (0, i), (0, i + 3))

    year_colors = {
        1: colors.HexColor("#e8f4fd"),
        2: colors.HexColor("#fdf4e8"),
        3: colors.HexColor("#e8fde8"),
        4: colors.HexColor("#f8e8fd"),
    }

    for i in range(1, len(table_data)):
        year_str = table_data[i][1]
        if "1st" in year_str: color = year_colors[1]
        elif "2nd" in year_str: color = year_colors[2]
        elif "3rd" in year_str: color = year_colors[3]
        elif "4th" in year_str: color = year_colors[4]
        else: continue
        style.add('BACKGROUND', (0, i), (-1, i), color)

    table.setStyle(style)
    doc.build([table])

In [14]:
# === Generate PDFs ===
generate_routine_pdf(term1_courses, "Routine_y1_t1.pdf")
#generate_routine_pdf(term2_courses, "Routine_y1_t2.pdf")